### 2. Setup

In [9]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://seller-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [10]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 3. We can get the credential by the credential_id used when storing

In [11]:
credential_id = "My Device Credential_0"
credential = await agent_controller.credentials.get_by_id(credential_id)
print(credential)

{'referent': 'My Device Credential_0', 'attrs': {'Manufacturer': 'GOOD_COMPANY', 'Type': 'Dentistry', 'ProductionDate': '1672981261', 'Status': '1', 'Owner': 'GOOD_SELLER', 'ExpirationDate': '1704517261'}, 'schema_id': 'T8ocwUDfMeVFzuVt5ExD2d:2:Devices_Schema:1.0.4', 'cred_def_id': 'T8ocwUDfMeVFzuVt5ExD2d:3:CL:78:default', 'rev_reg_id': 'T8ocwUDfMeVFzuVt5ExD2d:4:T8ocwUDfMeVFzuVt5ExD2d:3:CL:78:default:CL_ACCUM:c0921682-f9c0-4eac-869e-ef1d83e94bd8', 'cred_rev_id': '1'}


### 4. Setup Listener

In [12]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
          
proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)

Subscribing too: present_proof


### 5. Go to Manufacturer Notebook to send verify request

### 8. Get verify request

In [20]:
response = await agent_controller.proofs.get_records()
print(len(response['results']))

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID\n')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object\n')
print(response['results'][0]['presentation_request'])
print('Requested Attributes\n')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates\n')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

2


Presentation Exchange ID

716f734c-c2f4-4502-a48b-6b42069285fb
Presentation Request Object

{'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_ProductionDate_uuid': {'name': 'ProductionDate', 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}, '0_Manufacturer_uuid': {'name': 'Manufacturer', 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}, '0_Owner_uuid': {'name': 'Owner', 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}, '0_Type_uuid': {'name': 'Type', 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}}, 'requested_predicates': {'0_ExpirationDate_GE_uuid': {'name': 'ExpirationDate', 'p_type': '>=', 'p_value': 1672981450, 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}, '0_Status_GE_uuid': {'name': 'Status', 'p_type': '>', 'p_value': 0, 'restrictions': [{'issuer_did': 'T8ocwUDfMeVFzuVt5ExD2d'}]}}, 'non_revoked': {'to': 1672981450}, 'nonce': '837966065372533766383010'}
Requested Attributes

{'0_

### 9. Select credential from wallet

In [21]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "South Africa"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'My Device Credential_0', 'attrs': {'Owner': 'GOOD_SELLER', 'Type': 'Dentistry', 'ProductionDate': '1672981261', 'Status': '1', 'ExpirationDate': '1704517261', 'Manufacturer': 'GOOD_COMPANY'}, 'schema_id': 'T8ocwUDfMeVFzuVt5ExD2d:2:Devices_Schema:1.0.4', 'cred_def_id': 'T8ocwUDfMeVFzuVt5ExD2d:3:CL:78:default', 'rev_reg_id': 'T8ocwUDfMeVFzuVt5ExD2d:4:T8ocwUDfMeVFzuVt5ExD2d:3:CL:78:default:CL_ACCUM:c0921682-f9c0-4eac-869e-ef1d83e94bd8', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1672981450}, 'presentation_referents': ['0_ExpirationDate_GE_uuid', '0_ProductionDate_uuid', '0_Manufacturer_uuid', '0_Owner_uuid', '0_Status_GE_uuid', '0_Type_uuid']}]

Generate the proof
{'requested_predicates': {'0_ExpirationDate_GE_uuid': {'cred_id': 'My Device Credential_0'}, '0_Status_GE_uuid': {'cred_id': 'My Device Credential_0'}}, 'requested_attributes': {'0_ProductionDate_uuid': 

### 10. Send proof back

In [22]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'manufacturer': '30626903620271434916654302642552957304420863972179073023805462712935774458047', 'owner': '112342857966935099099520087438628370835770484748944694690714324726022568901624', 'productiondate': '1672981261', 'type': '16109800925107358751011721601810810012962164659284260983680380095017541118690'}, 'a_prime': '786502683297904389836011712959967172609409546616474776811381325058597478588732179285681316017507027339970388752440196457574246597570510983353524021486146437854072133342625557757273947150417456398831527336499421191667619525913130873839259185584490909008798451974500976549417733763600681656596566712409874149109286010173893098863532089200546957264618008721439656606753683533131672035502904416147907330172063227924203326949019444777513344308787431870886227894669079228981830845162565906731565551917888345570425270389614747951887750042737149526953196758995441306047243885011285697747586155428

### **Terminate**

In [8]:
response = await agent_controller.terminate()